In [1]:
import pandas as pd

ad = pd.read_csv('datasets/1976-2020-president.csv')
years = set(ad['year'])
states = set(ad['state'])

In [2]:
def to_percent(older, col_key):
    newer = []
    for y in years:
        for s in states:
            try:
                d = older.loc[y, s, 'DEMOCRAT'][col_key]
                r = older.loc[y, s, 'REPUBLICAN'][col_key]
                newer.append([int(y), s, 100*d/(d+r)]) # % share for each state
            except KeyError as e:
                pass
    return pd.DataFrame(data=newer, columns=['year', 'state', col_key, ])

In [3]:
pop_vote = pd.read_csv('datasets/1976-2020-president.csv')
pop_vote = pop_vote[['year', 'state', 'party_simplified', 'candidatevotes', ]] # drop useless columns
pop_vote = pop_vote.groupby(['year', 'state', 'party_simplified', ]).sum() # drop multiple candidate
pop_vote = to_percent(pop_vote, 'candidatevotes')
pop_vote.to_csv('datasets/pop_vote.csv', index=False)
pop_vote.head()

,year,state,candidatevotes
0,1984,ALASKA,30.944087
1,1984,MISSISSIPPI,37.684965
2,1984,LOUISIANA,38.580839
3,1984,ALABAMA,38.736605
4,1984,MONTANA,38.698601


In [4]:
a = pd.read_csv('datasets/gdp-77-97.csv')
b = pd.read_csv('datasets/gdp-97-20.csv')
gdp = []

# from row-column to table-list

for year in a.columns[2:]:
    for index, row in a.iterrows():
        gdp.append([int(year), row['GeoName'].upper(), row[year], ])

for year in b.columns[2:]:
    for index, row in b.iterrows():
        gdp.append([int(year), row['GeoName'].upper(), row[year], ])

gdp = [i for i in gdp if i[1] != 'NEW ENGLAND'] # remove new england
gdp = pd.DataFrame(data=gdp, columns=['year', 'state', 'gdp', ])
gdp.to_csv('datasets/gdp.csv', index=False)
gdp.head()

,year,state,gdp
0,1977,ALABAMA,59478.4
1,1977,ALASKA,16933.3
2,1977,ARIZONA,44588.9
3,1977,ARKANSAS,32633.1
4,1977,CALIFORNIA,523795.5


In [5]:
house_vote = pd.read_csv('datasets/1976-2020-house-utf8.csv')
house_vote = house_vote[['year', 'state', 'party', 'candidatevotes', ]]
house_vote = house_vote.groupby(['year', 'state', 'party', ]).sum()
house_vote = to_percent(house_vote, 'candidatevotes')
house_vote.head()
house_vote['year'] += 0
house_vote.to_csv('datasets/house_vote.csv', index=False)
house_vote.head()

,year,state,candidatevotes
0,1984,ALASKA,43.104882
1,1984,MISSISSIPPI,61.549294
2,1984,LOUISIANA,75.000000
3,1984,ALABAMA,72.726171
4,1984,MONTANA,51.190297


In [6]:
df = pd.merge(pop_vote, gdp, how='inner', left_on=['year','state'], right_on=['year','state'])
df = pd.merge(house_vote, df, how='inner', left_on=['year','state'], right_on=['year','state'])
df.head()

,year,state,candidatevotes_x,candidatevotes_y,gdp
0,1984,ALASKA,43.104882,30.944087,26853.7
1,1984,MISSISSIPPI,61.549294,37.684965,39379.1
2,1984,LOUISIANA,75.000000,38.580839,102250.7
3,1984,ALABAMA,72.726171,38.736605,70481.4
4,1984,MONTANA,51.190297,38.698601,15022.8


In [7]:
df.to_csv('datasets/xxx-final-dataset.csv', index=False)